# Flood Inundation Map accuracy Evaluation
## Created by: Sagy Cohen, University of Alabama; sagy.cohen@ua.edu
## Setting input data
### Input Layers:
#### Banchmark layer class: [0,2] 
#### Model Layer class: [1,2]

In [19]:
import numpy as np
arcpy.env.addOutputsToMap = False
arcpy.env.overwriteOutput = True

#Workspace (preferably a Geodatabase what include all the input layers):
ws = arcpy.env.workspace = r'C:\ModelEvalWorkshop\ModelEvalWorkshop_Workspace_1.gdb'

#Benchmark (remote sensing) FIM layer:
RSFIM = arcpy.Raster('FinalFIM_RSFlood1014EvalLayer_PWBremoved')

#Predicted (model) FIM layer:
#modelFIM = arcpy.Raster('HAND_1014') 
#modelFIM = arcpy.Raster('AutoRoute_1014')
modelFIM = arcpy.Raster('HECRAS_1014') 

### Calculating the comparison raster

In [20]:
#Map Algebra + of the model [1,2] and benchmark [0,2] rasters
#Values in the resulting raster indicate:
# 1 = True Negative (TN)
# 2 = False Positive (FP)
# 3 = False Negative (FN)
# 4 = True Positive (TP)

plusRas = RSFIM + modelFIM
print(modelFIM.name)
plusRas.save(modelFIM.name+ '_plus')

HECRAS_1014


In [21]:
#Exporting raster to Numpy Array for extracting the number of pixels in each category (TN, FP, FN, TP)
arr = arcpy.RasterToNumPyArray(plusRas)
valList = np.array(np.unique(arr, return_counts=True)).T.tolist()
print(valList)
TN,FP,FN,TP = 0,0,0,0
index_i = 0
for i in valList:
    if i[0] == 1: TN = valList[index_i][1]
    if i[0] == 2: FP = valList[index_i][1]
    if i[0] == 3: FN = valList[index_i][1]
    if i[0] == 4: TP = valList[index_i][1]
    index_i+=1
print(TN,FP,FN,TP)

[[1, 2305380], [2, 134820], [3, 89216], [4, 192685], [15, 749991]]
2305380 134820 89216 192685


### Calculating the evaluation metrics

In [22]:
CSI = TP/(TP+FP+FN)
TPR = TP/(TP+FN)
FAR = FP/(TP+FP)
F1 = (2*TP)/((2*TP)+FP+FN)
ACC = (TP+TN)/(TP+FP+TN+FN)
form = 'CSI,{0}\nTPR,{1}\nFAR,{2}\nF1,{3}\nACC,{4}'.format(CSI,TPR,FAR,F1,ACC)
print(form)

CSI,0.4623837051648465
TPR,0.6835201010283752
FAR,0.41165783728492694
F1,0.6323698814911569
ACC,0.917697396239155


### Saving the results to a CSV file

In [23]:
with open(r'C:\ModelEvalWorkshop\\' + RSFIM.name+'_'+modelFIM.name + '.csv', 'w') as f:
    f.write(form)